# Music Recommender

In [1]:
# import tools
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm.notebook import tqdm
from datetime import datetime
import random
from IPython.display import clear_output
from datetime import datetime

### Functions

In [2]:
   
# displays exit message
def say_goodbye():
    print('Thanks for using BEETz! Come back soon!')
    return



# checks for last scrape by reading txt file in which last date was stored
def check_last_scrape():
    with open('last_scrape.txt', 'r') as f:
        date = f.read()
    return date
        
    
    
# random excitement
def exclaim():
    exclamations=['SOO hot right now!', 'So hot its stolen!', 'Sizzzzzling!']
    exclamation = random.choice(exclamations)
    print('\n', exclamation)

    
    
# scrapes songs from online billboard, writes to csv, returns as df and records timestamp in txt file
def fetch_hot_songs():
    print('Fetching hot songs and artists from the web...')
    
    url = "https://www.billboard.com/charts/hot-100/"
    response = requests.get(url)
    website_data = response.text
    soup = BeautifulSoup(website_data, "html.parser")
    
    # append songs and artists to lists
    songs = []
    artists = []
    for i in tqdm(range(100)):
        songs.append(soup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
        artists.append(soup.select('span.c-label.a-no-trucate')[i].get_text(strip=True))
    
    # write to DataFrame
    df_dirty = pd.DataFrame({'songs':songs, 'artists':artists})
    df.to_csv('TopSongs.csv', index=False) 
    
    # write timestamp to txt file
    timestamp= datetime.today().strftime('%Y-%m-%d')
    with open('last_scrape.txt', 'w') as f:
        f.write(timestamp)
            
    print('Fetch completed!\n')
    return df
    
    
    
# gets random song from scraped billboard and checks if identical with requested song
def get_random_song(x, df):
    # takes sample from unclean dataframe
    sample = df.sample()
    rec_song = str(sample.iloc[0, 0])
    rec_artist = str(sample.iloc[0, 1])
    # clean song and check to make sure its not the same song as requested
    lower_song = rec_song.lower()
    song_cleaned = ''.join(e for e in lower_song if e.isalnum())
    if x in song_cleaned:
        get_random_song(x, df_clean, df)
    else:
        print(f'BEETz recommends {rec_song} by {rec_artist}!')
        print('Go again?')
    
        

        
# Based on run_search, asks user to enter song, formats and checks to see if song is in 'hot' songs
def song_search(df_clean, df):
    song = input('Please enter the title of a song you enjoy: \n').lower()
    # remove spaces and special characters
    song_cleaned = ''.join(e for e in song if e.isalnum())
    
    # check if song title is in top hits
    if df_clean['songs'].str.contains(song_cleaned).any():
        exclaim()
        return song_cleaned
    # if not, offer another search
    else:
        print('Your song doesn\'t seem to be very popular...try again? ')
        run_search(df_clean, df)
            
    
    
# Based on run_search, asks user to enter artist, formats and checks if artist is in 'hot' artists
def artist_search(df_clean, df):
    artist = input('Please enter the name of an artist you enjoy: \n').lower()
    # remove spaces and special characters
    artist_cleaned = ''.join(e for e in artist if e.isalnum())
    
    # check if artist is in top artists
    if df_clean['artists'].str.contains(artist_cleaned).any():
        exclaim()
        return '-'
    # if not, offer another search
    else:
        print('Your artist doesn\'t seem to be very popular...try again? ')
        run_search(df_clean, df)
            

            
# gets user search preference and executes search accordingly
def run_search(df_clean, df):    
    response = input('''
    Enter 1 to search by song.
    Enter 2 to search by artist. 
    Enter 3 to exit.\n\n''')  
    if response == '1':
        x = song_search(df_clean, df)
    elif response == '2':
        x = artist_search(df_clean, df)
    elif response == '3':
        say_goodbye()
        return
    else:
        print('Hm...that didn\'t work. Please try again.')
        run_search(df_clean, df)
    
    # Recommends random song after checking that it doesn't match searched song
    get_random_song(x, df)
    
    run_search(df_clean, df)
        

In [3]:
# the MOTHER function

def beetz():
    
    print('''
     ____  ______ ______ _______  
 |  _ \|  ____|  ____|__   __| 
 | |_) | |__  | |__     | |____
 |  _ <|  __| |  __|    | |_  /
 | |_) | |____| |____   | |/ / 
 |____/|______|______|  |_/___|
 ''')
        
    print('HELLO! Welcome to the BEETz song recommender!')
    print('BEETz will recommend a song related to the song or artist you enter.')
    
    
    # checks for last scrape - if not from today, launches new scrape
    if check_last_scrape() == datetime.today().strftime('%Y-%m-%d'):
        print('We have an up-to-date list of HOT songs from today on file!')
        df = pd.read_csv('TopSongs.csv', index_col=False)
    else:
        print('We will fetch an up-to-date list of HOT songs from today!')
        df = fetch_hot_songs()
    
    # lowercase and remove special characters and spaces from df
    df_lower = df.applymap(lambda s: s.lower() if type(s) == str else s)
    formatter = lambda s: ''.join(e for e in s if e.isalnum())
    df_clean = df_lower.applymap(formatter)
    
    # gets user search preference and executes search accordingly
    run_search(df_clean, df)
    

In [4]:
beetz()


     ____  ______ ______ _______  
 |  _ \|  ____|  ____|__   __| 
 | |_) | |__  | |__     | |____
 |  _ <|  __| |  __|    | |_  /
 | |_) | |____| |____   | |/ / 
 |____/|______|______|  |_/___|
 
HELLO! Welcome to the BEETz song recommender!
BEETz will recommend a song related to the song or artist you enter.
We have an up-to-date list of HOT songs from today on file!

    Enter 1 to search by song.
    Enter 2 to search by artist. 
    Enter 3 to exit.

3
Thanks for using BEETz! Come back soon!
